In [1]:
from dotenv import load_dotenv

from pathlib import Path
import os 
import sys
import numpy as np
import json

load_dotenv()
GEMINI_KEY = os.getenv('GEMINI_KEY')

root = Path().resolve().parent

root_str = str(root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)

if Path.cwd() != root:
    os.chdir(root)


### Generation

In [1]:
from google import genai

genai_client = genai.Client(
    api_key=GEMINI_KEY,
)

response = genai_client.models.generate_content(
    model="gemma-3-12b-it",
    contents="Roses are red...",
)

response.candidates[0].content.parts[0].text

NameError: name 'GEMINI_KEY' is not defined

### Troubleshoot invalid json

In [2]:
import json

string = "```json\n{\"api_name\":\"RegisterUser\",\"parameters\":{\"username\":\"johnsmith123\",\"password\":\"1234password\",\"email\":\"johnsmith123@example.com\"}}\n```"

json.loads(string)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
with open("data/generated/1-rotowire/generated.json", "r") as f:
    data = json.load(f)

json.loads(data[0]["generated_output"].replace("```json", "").replace("```", ""))

{'teams': [{'team': 'Atlanta Hawks',
   'total_points': 107,
   'points_in_2nd_quarter': 31,
   'percentage_of_field_goals': 49,
   'number_of_team_assists': 29,
   'wins': 9,
   'losses': 2},
  {'team': 'Milwaukee Bucks',
   'total_points': 100,
   'points_in_2nd_quarter': 9,
   'wins': 5,
   'losses': 5}],
 'players': [{'player': 'Giannis Antetokounmpo',
   'points': 26,
   'total_rebounds': 15,
   'assists': 7},
  {'player': 'Jabari Parker',
   'points': 23,
   'total_rebounds': 8,
   'assists': 3,
   'steals': 3},
  {'player': 'Tony Snell', 'points': 11, 'total_rebounds': 4, 'assists': 3},
  {'player': 'Mirza Teletovic',
   'points': 14,
   'field_goals_made': 5,
   'field_goals_attempted': 10},
  {'player': 'Paul Millsap', 'points': 21, 'total_rebounds': 8, 'assists': 3},
  {'player': 'Mike Muscala', 'points': 16, 'total_rebounds': 5, 'assists': 3},
  {'player': 'Kyle Korver', 'points': 11, 'total_rebounds': 6, 'assists': 5}]}

In [5]:
from src.evaluate import assess_json_valid

s = """```json
{
    'article_title': "Bibs Ekkel",
    'associated_acts': "tziganka; balalaika potpourri; russian tornado",
    'background': "mixed polish-english",
    'birth_date': "1946-01-11",
    'birth_name': "",
    'genre': "russian folk music",
    'instrument': "balalaika",
    'name': "Bibs Ekkel",
    'occupation': "balalaika player; lecturer; music adviser",
    'origin': "brighton",
    'years_active': "19751946"
}
```"""

assess_json_valid(s)

Invalid JSON string: ```json
{
    'article_title': "Bibs Ekkel",
    'associated_acts': "tziganka; balalaika potpourri; russian tornado",
    'background': "mixed polish-english",
    'birth_date': "1946-01-11",
    'birth_name': "",
    'genre': "russian folk music",
    'instrument': "balalaika",
    'name': "Bibs Ekkel",
    'occupation': "balalaika player; lecturer; music adviser",
    'origin': "brighton",
    'years_active': "19751946"
}
``` - Fixing


(0,
 {'article_title': 'Bibs Ekkel',
  'associated_acts': 'tziganka; balalaika potpourri; russian tornado',
  'background': 'mixed polish-english',
  'birth_date': '1946-01-11',
  'birth_name': '',
  'genre': 'russian folk music',
  'instrument': 'balalaika',
  'name': 'Bibs Ekkel',
  'occupation': 'balalaika player; lecturer; music adviser',
  'origin': 'brighton',
  'years_active': '19751946'})

In [ ]:
import json

s = """```json
{
    'article_title': "Bibs Ekkel",
    'associated_acts': "tziganka; balalaika potpourri; russian tornado",
    'background': "mixed polish-english",
    'birth_date': "1946-01-11",
    'birth_name': "",
    'genre': "russian folk music",
    'instrument': "balalaika",
    'name': "Bibs Ekkel",
    'occupation': "balalaika player; lecturer; music adviser",
    'origin': "brighton",
    'years_active': "19751946"
}
```"""

a = s.replace("```json", "").replace("```", "")
json.loads(a)

JSONDecodeError: Expecting property name enclosed in double quotes: line 3 column 5 (char 7)

In [10]:
print(a)
json.loads(a)


{
    'article_title': "Bibs Ekkel",
    'associated_acts': "tziganka; balalaika potpourri; russian tornado",
    'background': "mixed polish-english",
    'birth_date': "1946-01-11",
    'birth_name': "",
    'genre': "russian folk music",
    'instrument': "balalaika",
    'name': "Bibs Ekkel",
    'occupation': "balalaika player; lecturer; music adviser",
    'origin': "brighton",
    'years_active': "19751946"
}



JSONDecodeError: Expecting property name enclosed in double quotes: line 3 column 5 (char 7)

# llama-cpp

In [8]:
prompt = """You role is to answer a math question.    
You will always repond with JSON in the format described below:

{"reasoning": <reasoning about the answer>, "answer": <final answer>}

Do not include anything apart from the json, no extra text or markdown.
The "reasoning" field will contain your reasoning about the sequence of events, and the "answer" will contain only the answer, in the form of an integer."""


record = {
    "id": 407,
    "input": "Elliott is trying to walk 10,000 steps a day. He finished half of his steps on his walks to and from school and did another 1,000 steps going for a short walk with his friend. He also went for a short jog around the block and realized that after he had finished his jog, he only had 2,000 steps left to take. How many steps did Elliott take during his jog?",
    "output": {
      "reasoning": "Elliott took half of his steps on his school journey, which was 10,000 steps / 2 = <<10000/2=5000>>5,000 steps.\nAdding this to his short walk means he has taken 5,000 + 1,000 = <<6000=6000>>6,000 steps.\nHe only has 2,000 steps left, which means that during the day, he has taken 10,000 goal steps \u2013 2,000 remaining steps = <<10000-2000=8000>>8,000 steps.\nThe difference between these is therefore the steps he took during his jog, which was 8,000 total steps \u2013 6,000 steps on his school walks and short walks = <<8000-6000=2000>>2,000 steps.",
      "answer": 2000
    }
  }

In [6]:
from llama_cpp import Llama


llm = Llama(
    model_path="models/Qwen3-1.7B-Q8_0.gguf",
    chat_format="qwen3",
    temperature=0.0,    # disable temperature-based randomness
    top_k=1,            # only look at the single highest-probability token
    top_p=1.0           # (optional) allow the full distribution, but top_k=1 already constrains it
)

messages =  [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user",   "content": prompt + "\n\nInput: " + record['input'] + "\nOutput:"},
]
resp = llm.create_chat_completion(messages, max_tokens=1024)

print(resp["choices"][0]["text"])

llama_model_load_from_file_impl: using device Metal (Apple M2 Pro) - 21839 MiB free
llama_model_loader: loaded meta data with 28 key-value pairs and 310 tensors from models/Qwen3-1.7B-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen3 1.7B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Qwen3
llama_model_loader: - kv   5:                         general.size_label str              = 1.7B
llama_model_loader: - kv   6:                          qwen3.block_count u32 

LlamaChatCompletionHandlerNotFoundException: Invalid chat handler: qwen3 (valid formats: ['llama-2', 'llama-3', 'alpaca', 'qwen', 'vicuna', 'oasst_llama', 'baichuan-2', 'baichuan', 'openbuddy', 'redpajama-incite', 'snoozy', 'phind', 'intel', 'open-orca', 'mistrallite', 'zephyr', 'pygmalion', 'chatml', 'mistral-instruct', 'chatglm3', 'openchat', 'saiga', 'gemma', 'functionary', 'functionary-v2', 'functionary-v1', 'chatml-function-calling'])

In [41]:
resp["choices"][0]["message"]["content"]

'<think>\nOkay, let\'s try to figure out how many steps Elliott took during his jog. \n\nFirst, Elliott wants to walk 10,000 steps a day. He did half of his steps on his walks to and from school. Wait, half of his total steps? Or half of the distance to and from school? Hmm, the wording is a bit tricky. Let me read it again: "He finished half of his steps on his walks to and from school." So, when he goes to school and comes back, he did half of his total steps for the day during that walk. \n\nSo, if he walks to and from school, that\'s a round trip. Let\'s assume each way is a certain distance, but maybe the key is that he did half of his total steps during that walk. Let me think. Let\'s say the total steps he needs to take in the day is 10,000. He did half of that during his school walk. So half of 10,000 is 5,000 steps. So he took 5,000 steps during the school walk. \n\nThen he did another 1,000 steps going for a short walk with his friend. So that\'s 1,000 steps. Then he went for

In [34]:
messages = messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user",   "content": "Hey!"}
]
resp = llm.create_chat_completion(messages, max_tokens=1024)

print(resp["choices"][0]["text"])

LlamaChatCompletionHandlerNotFoundException: Invalid chat handler: qwen3 (valid formats: ['llama-2', 'llama-3', 'alpaca', 'qwen', 'vicuna', 'oasst_llama', 'baichuan-2', 'baichuan', 'openbuddy', 'redpajama-incite', 'snoozy', 'phind', 'intel', 'open-orca', 'mistrallite', 'zephyr', 'pygmalion', 'chatml', 'mistral-instruct', 'chatglm3', 'openchat', 'saiga', 'gemma', 'functionary', 'functionary-v2', 'functionary-v1', 'chatml-function-calling'])

In [31]:
resp = llm.create_completion("hey!", max_tokens=256)
print(resp["choices"][0]["text"])

Llama.generate: 1 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     712.27 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    4990.57 ms /   256 runs   (   19.49 ms per token,    51.30 tokens per second)
llama_perf_context_print:       total time =    5091.13 ms /   257 tokens


 I want to create a simple app in python with tkinter, that can do the following:

- has a button to switch between two modes, say "normal" and "dark"
- when the button is clicked, the app changes the color of the background and the text
- has a toggle button that can switch between two different modes, say, "dark" and "light"
- when the button is clicked, the app changes the color of the background and the text
- has a toggle button that can switch between two different modes, say, "dark" and "light"

Wait, no, the user is saying that the app has two modes, "normal" and "dark", and then a toggle button that can switch between "dark" and "light". So it's a two-level toggle.

So the app should have a button to switch between normal and dark mode. Then, there is another toggle button that switches between dark and light. So the app has two toggle switches, and when the user clicks the first button, the app changes the color from normal to dark, and then when the user clicks the second bu

### Transformers

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen3-1.7B"        # or any other Qwen3 size
MODEL_DIR = Path("models")

# 1 Load
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=MODEL_DIR)
model      = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",               # BF16 / FP16 on GPU, FP32 on CPU
    device_map="auto",                 # lets Accelerate pick the best device(s),
    cache_dir=MODEL_DIR
)

# 2 Build a list of chat messages
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user",   "content": prompt + "\n\nInput: " + record['input'] + "\nOutput:"}
]

# 3 Convert the list to the *exact* prompt string the model expects
prompt_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,                   # we want a plain string
    add_generation_prompt=True,       # adds the assistant-turn marker
    enable_thinking=False             # = non-thinking / normal chat
    # enable_thinking=True            # uncomment for chain-of-thought mode
)

# 4 Tokenise & run generation
inputs = tokenizer([prompt_text], return_tensors="pt").to(model.device)
gen_ids = model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.7,                  # recommended for non-thinking mode
    top_p=0.8,
    top_k=20
)
reply = tokenizer.decode(gen_ids[0][inputs.input_ids.shape[-1]:],
                         skip_special_tokens=True)
print(reply)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

{"reasoning": "Elliott walked to and from school, which is 10,000 steps total. He finished half of his steps, so he took 5,000 steps for the walk to and from school. He then took an additional 1,000 steps for a short walk with his friend. After his jog, he had 2,000 steps left to take. Therefore, the number of steps he took during his jog is 10,000 - 5,000 - 1,000 - 2,000 = 2,000.", "answer": 2000}


In [11]:
model.device

device(type='mps', index=0)

### prompt eng

In [17]:
from pathlib import Path
import shutil

def copy_prompt(folder: Path | str = Path("data/clean")) -> None:
    folder = Path(folder)

    # Look for prompt.txt directly inside this folder
    prompt_file = folder / "prompt.txt"
    if prompt_file.exists():
        for suffix in ("_low", "_high"):
            shutil.copyfile(prompt_file, folder / f"prompt{suffix}.txt")

    # Recurse into sub-directories
    for sub in folder.iterdir():
        if sub.is_dir():
            copy_prompt(sub)

# run it
copy_prompt()

### solve gemma 12 pb

In [5]:
from src.generate_gemma import process_batch
from google import genai

client = genai.client.Client(api_key=GEMINI_KEY)

data_path = Path("data/clean/5-api_bank")
output_path = Path("results/gemma-3-12b-it/5-api_bank/temp")
model_name = "gemma-3-12b-it"
prompt_type = None

bench_file = "results/gemma-3-12b-it/5-api_bank/temp.json"
prompt_file = data_path / f'prompt{"_" + prompt_type if prompt_type else ""}.txt'

with open(bench_file, 'r') as f:
    bench_data = json.load(f)

with open(prompt_file, 'r') as f:
    prompt_template = f.read()

prompts = [
        (prompt_template
        .replace('{input}', record['input'])
        .replace('{instruction}', record['instruction']))
    for record in bench_data
]
process_batch(client, prompts, bench_data, output_path, "5-api_bank", model_name)

Generating for 5-api_bank: 100%|██████████| 2/2 [00:02<00:00,  1.18s/it]
